In [3]:
# Importamos las librerías necesarias
import pandas as pd
#import geopandas
#import shapely.wkt
#import rtree
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 100
%matplotlib inline
pd.set_option("display.max_rows", 101)

In [4]:
# Levantamos el dataset
propiedades_original = pd.read_csv("properatti.csv"
           , encoding = 'utf-8')

In [5]:
propiedades = propiedades_original

In [6]:
# Consultamos si hay duplicados por la columna description y si hay, los borramos
canti = propiedades.duplicated(subset='description').sum()
if canti != 0:
    print("Existen ", canti, " filas duplicadas de acuerdo a su descripción, se eliminan.")
    propiedades = propiedades.drop_duplicates(subset='description', keep="last")

Existen  17164  filas duplicadas de acuerdo a su descripción, se eliminan.


In [7]:
propiedades = propiedades.drop(propiedades[propiedades.state_name!='Capital Federal'].index)

In [8]:
propiedades.columns.values 

array(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name',
       'geonames_id', 'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd',
       'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2',
       'price_per_m2', 'floor', 'rooms', 'expenses', 'properati_url',
       'description', 'title', 'image_thumbnail'], dtype=object)

In [16]:
# Reemplazo los NAN de la superficie
propiedades['surface_total_in_m2']=np.where(propiedades["surface_total_in_m2"].isnull(), propiedades['surface_covered_in_m2'],propiedades["surface_total_in_m2"])
propiedades['surface_total_in_m2']=np.where(propiedades["surface_total_in_m2"].isnull(), 0,propiedades["surface_total_in_m2"])
pd.isna(propiedades.surface_total_in_m2).sum()

0

In [17]:
# CAMBIO DE VALORES NAN DE PLACE_NAME
# Aunque la ubicacion no aparece escrita, si aparecen las coordenadas
# Estas coordenadas pertenecen a Nordelta por lo que procedemos a cambiar el place name por Nordelta  

propiedades['place_name'] = propiedades['place_name'].replace(np.nan, 'Nordelta')


In [18]:
# Revisamos las medianas por barrio y tipo de propiedad
propiedades2 = propiedades[['place_name', 'property_type','price_aprox_usd']].groupby(['place_name', 'property_type']).aggregate([np.median])
propiedades2

price_aprox_usd
                                            median
place_name           property_type                
Abasto               PH                  160000.00
                     apartment           127500.00
                     house               490000.00
                     store               380000.00
Agronomía            PH                  220000.00
                     apartment           130000.00
                     house               432500.00
                     store               420000.00
Almagro              PH                  220000.00
                     apartment           125000.00
                     house               450000.00
                     store               192500.00
Balvanera            PH                  150000.00
                     apartment           113000.00
                     house               248500.00
                     store               340000.00
Barracas             PH                  150000.00
                     apartment           132050.00
                     house               260000.00
                     store               265000.00
Barrio Norte         PH                  187500.00
                     apartment           225000.00
                     house               895000.00
                     store               225000.00
Belgrano             PH                  294500.00
                     apartment           215356.20
                     house               845000.00
                     store               209000.00
Boca                 PH                  134950.00
                     apartment            92000.00
                     house               216000.00
                     store               160000.00
Boedo                PH                  155000.00
                     apartment           222500.00
                     house               350000.00
                     store               185000.00
Caballito            PH                  199000.00
                     apartment           140000.00
                     house               400000.00
                     store               181755.50
Capital Federal      PH                  150000.00
                     apartment           140000.00
                     house               495000.00
                     store               272000.00
Catalinas            apartment           550000.00
                     store               350000.00
Centro / Microcentro apartment           107981.00
                     house               820000.00
                     store               270000.00
Chacarita            PH                  191500.00
...                                            ...
Versalles            house               303500.00
Villa Crespo         PH                  163500.00
                     apartment           125929.00
                     house               460000.00
                     store               250000.00
Villa Devoto         PH                  173500.00
                     apartment           125000.00
                     house               358000.00
                     store               235000.00
Villa General Mitre  PH                  197500.00
                     apartment            99500.00
                     house               320000.00
Villa Lugano         PH                  125000.00
                     apartment            76500.00
                     house               195000.00
                     store                80000.00
Villa Luro           PH                  149000.00
                     apartment           113500.00
                     house               320000.00
                     store               347500.00
Villa Ortuzar        PH                  230000.00
                     apartment           107000.00
                     house               410000.00
                     store               370000.00
Villa Pueyrredón     PH                  193950.00
      

In [19]:
# Reemplazamos por la mediana en los campos cuyo precio en dólares sea NAN (este paso tarda unos minutos)
def completa_nan(valor, str_place_name,str_property_type):
    df2 = propiedades.loc[(propiedades['place_name'] == str_place_name) & (propiedades['property_type'] == str_property_type)]
    return df2.price_aprox_usd.median()

# Revisamos que haya nulos
canti = propiedades["price_aprox_usd"].isnull().sum()
print(canti)
if canti != 0:
    propiedades["price_aprox_usd"] = propiedades["price_aprox_usd"].apply(lambda X: completa_nan(X, propiedades.place_name, propiedades.property_type) if np.isnan(X) else X)

# Revisamos que no haya nulos
canti = propiedades["price_aprox_usd"].isnull().sum()
canti

1884


0

In [20]:
# Testeamos los outliers del price_aprox_usd
def drop_outliers(df, field_name):
    distance = 1.5 * (np.percentile(df[field_name], 75) - np.percentile(df[field_name], 25))
    df.drop(df[df[field_name] > distance + np.percentile(df[field_name], 75)].index, inplace=True)
    df.drop(df[df[field_name] < np.percentile(df[field_name], 25) - distance].index, inplace=True)
    
drop_outliers(propiedades,'price_aprox_usd')
drop_outliers(propiedades,'surface_total_in_m2')

In [21]:
# Borramos columnas que no vamos a usar para una mejor visualización del dataset
propiedades.drop(['Unnamed: 0'], axis=1, inplace=True)
propiedades.drop(['operation'], axis=1, inplace=True)
propiedades.drop(['place_with_parent_names'], axis=1, inplace=True)
propiedades.drop(['country_name'], axis=1, inplace=True)
propiedades.drop(['state_name'], axis=1, inplace=True)
propiedades.drop(['geonames_id'], axis=1, inplace=True)
propiedades.drop(['lat-lon'], axis=1, inplace=True)
propiedades.drop(['lat'], axis=1, inplace=True)
propiedades.drop(['lon'], axis=1, inplace=True)
propiedades.drop(['properati_url'], axis=1, inplace=True)
propiedades.drop(['image_thumbnail'], axis=1, inplace=True)
propiedades.head(100)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,title
0,PH,Mataderos,62000.0,USD,1093959.00,62000.00,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
2,apartment,Mataderos,72000.0,USD,1270404.00,72000.00,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,PH,Liniers,95000.0,USD,1676227.50,95000.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado
7,apartment,Belgrano,138000.0,USD,2434941.00,138000.00,45.0,40.0,3066.666667,3450.000000,NaN,NaN,NaN,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...
8,apartment,Belgrano,195000.0,USD,3440677.50,195000.00,65.0,60.0,3000.000000,3250.000000,NaN,NaN,NaN,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,..."
13,apartment,Palermo Soho,111700.0,USD,1970890.65,111700.00,50.0,30.0,2234.000000,3723.333333,NaN,1.0,NaN,Torre I Mondrian. 3 ambientes con terraza y d...,Vitraux Palermo
14,apartment,Palermo Soho,147900.0,USD,2609621.55,147900.00,42.0,31.0,3521.428571,4770.967742,NaN,1.0,NaN,Torre II Dalí. Ambiente unico divisible.Vitrau...,Vitraux Palermo
16,PH,Mataderos,239000.0,USD,4217035.50,239000.00,140.0,98.0,1707.142857,2438.775510,NaN,4.0,NaN,Ventas Mataderos al frente Duplex 4 amb.- Plan...,VENTA-MATADEROS-DUPLEX 4 AMB.
19,apartment,Palermo,350000.0,USD,6175575.00,350000.00,104.0,96.0,3365.384615,3645.833333,NaN,3.0,NaN,Excelente semipiso al contra frente en Bulnes ...,"Bulnes y Libertador: espectacular pulmón, con ..."
21,apartment,Palermo,270500.0,USD,4772837.25,270500.00,118.0,73.0,2292.372881,3705.479452,NaN,4.0,NaN,"EXCELENTE ZONA, MULTIPLES MEDIOS DE TRANSPORTE...",Departamento de 4 ambientes en Venta en Palermo


In [22]:
# Se analizan los NAN de la columna rooms
print(propiedades['rooms'].isnull().sum())

# Lo intentamos obtener del título y la descripción con expresiones regulares
rooms = propiedades["title"].str.extract("(\d+)\s?AMB+",expand=True)
rooms = rooms.fillna(propiedades["title"].str.extract("(\d+)\s?DORM+",expand=True))
rooms = rooms.fillna(propiedades["title"].str.extract("(\d+)\s?amb+",expand=True))
rooms = rooms.fillna(propiedades["title"].str.extract("(\d+)\s?dorm+",expand=True))
rooms = rooms.fillna(propiedades["description"].str.extract("(\d+)\s?AMB+",expand=True))
rooms = rooms.fillna(propiedades["description"].str.extract("(\d+)\s?DORM+",expand=True))
rooms = rooms.fillna(propiedades["description"].str.extract("(\d+)\s?amb+",expand=True))
rooms = rooms.fillna(propiedades["description"].str.extract("(\d+)\s?dorm+",expand=True))

# La paso a float como rooms
rooms=rooms.astype('float64')

# Agrego la nueva columna
propiedades['rooms2'] = rooms

#Uso la columna nueva para completar y luego la borro
propiedades['rooms']=np.where(propiedades["rooms"].isnull(), propiedades['rooms2'],propiedades["rooms"])
# Borro la columna nueva
propiedades.drop(['rooms2'], axis=1, inplace=True)

# Veo cuantos null quedan
print(propiedades['rooms'].isnull().sum())

10500
4695


In [23]:
# Estos null los vamos a inferir de la superficie cubierta de la propiedad
# Reglas: 
# Hasta 30 m2 son monoambientes, 0 habitaciones
# Hasta 50 m2 son 2 ambientes, 1 habitación
# Hasta 70 m2 son 3 ambientes, 2 habitaciones
# Hasta 90 m2 son 4 ambientes, 3 habitaciones
# Hasta 110 m2 son 5 ambientes, 4 habitaciones
# Más de 110 m2 son 6 ambientes, 5 habitaciones

# Función que devuelve los ambientes
def setea_ambientes(sup_cub):
    if (sup_cub <= 30):
        return 1
    else:
        if (sup_cub <= 50):
            return 2
        else:
            if (sup_cub <= 70):
                return 3
            else:
                if (sup_cub <= 90):
                    return 4
                else:
                    if (sup_cub <= 110):
                        return 5
                    else:
                        if (sup_cub > 110):
                            return 6
                        else:
                            return 0
        
# Llamo a la función con un lambda para una nueva columna
propiedades["rooms2"] = propiedades["surface_covered_in_m2"].apply(lambda X: setea_ambientes(X))
# Uso la columna nueva
propiedades['rooms']=np.where(propiedades["rooms"].isnull(), propiedades['rooms2'],propiedades["rooms"])
# Borro la columna nueva
propiedades.drop(['rooms2'], axis=1, inplace=True)

# Veo cuantos null quedan
print(propiedades['rooms'].isnull().sum())

# Veo cuantas propiedades hay por cantidad de ambientes
from collections import Counter
print(Counter(propiedades['rooms']))
print (propiedades.shape)

# Elimino outliers -> propiedades con más de 6 ambientes
propiedades.drop(propiedades[propiedades.rooms > 6].index, inplace=True)
print(Counter(propiedades['rooms']))

print (propiedades.shape)

0
Counter({2.0: 7098, 3.0: 5452, 1.0: 3990, 4.0: 3272, 5.0: 750, 0.0: 546, 6.0: 411, 7.0: 41, 8.0: 12, 10.0: 5, 9.0: 3, 2017.0: 1, 11.0: 1, 18.0: 1, 4000.0: 1, 36.0: 1, 12.0: 1})
(21586, 15)
Counter({2.0: 7098, 3.0: 5452, 1.0: 3990, 4.0: 3272, 5.0: 750, 0.0: 546, 6.0: 411})
(21519, 15)


In [24]:
# Definimos una función que dado un texto lo normaliza para hacer más fácil la generación de columnas nuevas
def normalizar(str_x):    
    if isinstance(str_x, str):
        str_x = str_x.lower()
        str_x = str_x.replace(" amb "," ambientes ")
        str_x = str_x.replace(" amb. "," ambientes ")
        str_x = str_x.replace(" ambientesientes "," ambientes ")
        str_x = str_x.replace(" depto "," departamento ")
        str_x = str_x.replace(" depto. "," departamento ")
        str_x = str_x.replace(" dorm "," dormitorio ")
        str_x = str_x.replace(" dorm. "," dormitorio ")
        str_x = str_x.replace(" dormitorioss "," dormitorios ")
        str_x = str_x.replace(" bano "," baño ")
        str_x = str_x.replace(" banio "," baño ")
        str_x = str_x.replace(" banios "," baño ")
        str_x = str_x.replace(" banos "," baños ")
        str_x = str_x.replace(" garage "," cochera ")
        str_x = str_x.replace(" laundry "," lavadero ")
        str_x = str_x.replace(" piscina "," pileta ")
        str_x = str_x.replace(" apto credito "," apto crédito ")

    return str_x

# Invocamos la función para las columnas description y title
propiedades["description"] = propiedades.description.apply(normalizar)
propiedades["title"] = propiedades.title.apply(normalizar)

In [25]:
# Generamos una función genérica para buscar textos y devolver la cantidad de ocurrencias
def buscar_con_canti(X, str_busqueda):
    if isinstance(X, str):
        #return str_busqueda in X
        return sum([1 for i in X.split() if str_busqueda in i.lower()])
    else:
        return 0
    
# Generamos una función genérica para buscar textos y devolver true o false si aparece
def buscar_sin_canti(X, str_busqueda):
    if isinstance(X, str):
        return str_busqueda in X
    else:
        return False
    

In [26]:
# Intentamos obtener la cantidad de baños
propiedades["Baño"] = propiedades.description.apply(buscar_con_canti,str_busqueda='baño')

# Intentamos saber si tiene cochera
propiedades["Cochera"] = propiedades.description.apply(buscar_sin_canti,str_busqueda='cochera')

# Intentamos saber si tiene pileta
propiedades["Pileta"] = propiedades.description.apply(buscar_sin_canti,str_busqueda='pileta')

# Intentamos saber si tiene dependencia
propiedades["Con dependencia"] = propiedades.description.apply(buscar_sin_canti,str_busqueda='con dependencia')

# Intentamos saber si tiene patio
propiedades["Patio"] = propiedades.description.apply(buscar_sin_canti,str_busqueda='patio')

# Intentamos saber si es apto crédito
propiedades["Apto crédito"] = propiedades.description.apply(buscar_sin_canti,str_busqueda='apto crédito')

# Intentamos saber si es luminoso
propiedades["Luminoso"] = propiedades.description.apply(buscar_sin_canti,str_busqueda='luminoso')

propiedades.head(12)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,...,expenses,description,title,Baño,Cochera,Pileta,Con dependencia,Patio,Apto crédito,Luminoso
0,PH,Mataderos,62000.0,USD,1093959.00,62000.0,55.0,40.0,1127.272727,1550.000000,...,NaN,"2 ambientes tipo casa planta baja por pasillo,...",2 ambientes tipo casa sin expensas en pb,1,False,False,False,True,True,False
2,apartment,Mataderos,72000.0,USD,1270404.00,72000.0,55.0,55.0,1309.090909,1309.090909,...,NaN,2 ambientes 3er piso lateral living comedor am...,2 ambientes 3er piso con ascensor apto credito,1,False,False,False,False,False,False
3,PH,Liniers,95000.0,USD,1676227.50,95000.0,0.0,NaN,NaN,NaN,...,NaN,ph 3 ambientes con patio. hay 3 deptos en lote...,ph 3 ambientes cfte. reciclado,0,False,False,False,True,False,False
7,apartment,Belgrano,138000.0,USD,2434941.00,138000.0,45.0,40.0,3066.666667,3450.000000,...,NaN,excelente monoambiente a estrenar amplio super...,jose hernandez 1400 monoambiente estrenar cat...,1,False,True,False,False,False,True
8,apartment,Belgrano,195000.0,USD,3440677.50,195000.0,65.0,60.0,3000.000000,3250.000000,...,NaN,excelente dos ambientes estrenar amplio super...,"jose hernandez 1400 dos ambientes estrenar ,...",1,False,True,False,False,False,True
13,apartment,Palermo Soho,111700.0,USD,1970890.65,111700.0,50.0,30.0,2234.000000,3723.333333,...,NaN,torre i mondrian. 3 ambientes con terraza y d...,vitraux palermo,1,False,True,False,False,False,False
14,apartment,Palermo Soho,147900.0,USD,2609621.55,147900.0,42.0,31.0,3521.428571,4770.967742,...,NaN,torre ii dalí. ambiente unico divisible.vitrau...,vitraux palermo,0,False,True,False,False,False,False
16,PH,Mataderos,239000.0,USD,4217035.50,239000.0,140.0,98.0,1707.142857,2438.775510,...,NaN,ventas mataderos al frente duplex 4 amb.- plan...,venta-mataderos-duplex 4 amb.,1,False,False,False,False,False,False
19,apartment,Palermo,350000.0,USD,6175575.00,350000.0,104.0,96.0,3365.384615,3645.833333,...,NaN,excelente semipiso al contra frente en bulnes ...,"bulnes y libertador: espectacular pulmón, con ...",1,False,False,False,False,False,True
21,apartment,Palermo,270500.0,USD,4772837.25,270500.0,118.0,73.0,2292.372881,3705.479452,...,NaN,"excelente zona, multiples medios de transporte...",departamento de 4 ambientes en venta en palermo,0,False,False,False,True,False,False


In [27]:
# Dummies de la zona
place_name_dummies = pd.get_dummies(propiedades.place_name, prefix='place_name',drop_first='true')
place_name_dummies.sample(n=5, random_state=1)

,place_name_Agronomía,place_name_Almagro,place_name_Balvanera,place_name_Barracas,place_name_Barrio Norte,place_name_Belgrano,place_name_Boca,place_name_Boedo,place_name_Caballito,place_name_Capital Federal,...,place_name_Villa Lugano,place_name_Villa Luro,place_name_Villa Ortuzar,place_name_Villa Pueyrredón,place_name_Villa Real,place_name_Villa Riachuelo,place_name_Villa Santa Rita,place_name_Villa Soldati,place_name_Villa Urquiza,place_name_Villa del Parque
11343,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54710,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
76977,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
95163,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
119688,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [28]:
# Dummies del tipo de propiedad
property_type_dummies = pd.get_dummies(propiedades.property_type, prefix='property_type',drop_first='true')
property_type_dummies.sample(n=5, random_state=1)

,property_type_apartment,property_type_house,property_type_store
11343,1,0,0
54710,0,0,0
76977,1,0,0
95163,1,0,0
119688,0,0,0


In [29]:
# Dummies de cochera
cochera_dummies = pd.get_dummies(propiedades.Cochera, prefix='cochera',drop_first='true')
cochera_dummies.sample(n=5, random_state=1)

,cochera_True
11343,0
54710,0
76977,1
95163,1
119688,0


In [30]:
# Dummies de pileta
pileta_dummies = pd.get_dummies(propiedades.Pileta, prefix='Pileta',drop_first='true')
pileta_dummies.sample(n=5, random_state=1)

,Pileta_True
11343,0
54710,0
76977,0
95163,0
119688,0


In [31]:
# Dummies de dependencia
dependencia_dummies = pd.get_dummies(propiedades["Con dependencia"], prefix='Dependencia',drop_first='true')
dependencia_dummies.sample(n=5, random_state=1)

,Dependencia_True
11343,0
54710,0
76977,0
95163,0
119688,0


In [32]:
# Dummies de patio
patio_dummies = pd.get_dummies(propiedades["Patio"], prefix='Patio',drop_first='true')
patio_dummies.sample(n=5, random_state=1)

,Patio_True
11343,0
54710,0
76977,0
95163,0
119688,1


In [33]:
# Dummies de apto crédito
apto_dummies = pd.get_dummies(propiedades["Apto crédito"], prefix='apto',drop_first='true')
apto_dummies.sample(n=5, random_state=1)

,apto_True
11343,0
54710,0
76977,0
95163,0
119688,1


In [34]:
# Dummies de luminoso
luminoso_dummies = pd.get_dummies(propiedades["Luminoso"], prefix='apto',drop_first='true')
luminoso_dummies.sample(n=5, random_state=1)

,apto_True
11343,0
54710,0
76977,0
95163,0
119688,0


In [35]:
propiedades.columns.values

array(['property_type', 'place_name', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd',
       'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2',
       'price_per_m2', 'floor', 'rooms', 'expenses', 'description',
       'title', 'Baño', 'Cochera', 'Pileta', 'Con dependencia', 'Patio',
       'Apto crédito', 'Luminoso'], dtype=object)

In [38]:
propiedades_prediction = pd.concat([place_name_dummies, property_type_dummies], axis=1)
propiedades_prediction = pd.concat([propiedades_prediction, apto_dummies], axis=1)
propiedades_prediction = pd.concat([propiedades_prediction, pileta_dummies, cochera_dummies, dependencia_dummies, patio_dummies], axis=1) 
propiedades_prediction = pd.concat([propiedades_prediction, luminoso_dummies], axis=1)
propiedades_prediction = pd.concat([propiedades_prediction, propiedades.rooms, propiedades.surface_total_in_m2], axis=1)
propiedades_prediction.head()

,place_name_Agronomía,place_name_Almagro,place_name_Balvanera,place_name_Barracas,place_name_Barrio Norte,place_name_Belgrano,place_name_Boca,place_name_Boedo,place_name_Caballito,place_name_Capital Federal,...,property_type_house,property_type_store,apto_True,Pileta_True,cochera_True,Dependencia_True,Patio_True,apto_True,rooms,surface_total_in_m2
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,2.0,55.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.0,55.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,3.0,0.0
7,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,2.0,45.0
8,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,3.0,65.0


In [39]:
# Serializamos
propiedades_prediction.to_pickle('propiedades_prediction.pkl')
propiedades.to_pickle('propiedades.pkl')